# Credits to [Sidhant](https://www.kaggle.com/thedocs) for the pre-processing !!

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch
from torchvision import models,transforms
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from pickle import dump
from torch.utils.data import Dataset,DataLoader
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
device = torch.device('cpu')
data = pd.read_csv("../input/jobposts/data job posts.csv")
data

In [ ]:
data.RequiredQual

In [ ]:
data.columns

In [ ]:
data.JobDescription

In [ ]:
data.JobRequirment

In [ ]:
df = data[["RequiredQual","JobDescription","JobRequirment","Title"]].dropna()
df

In [ ]:
classes = df['Title'].value_counts()[:20]
keys = classes.keys().to_list()

df = df[df['Title'].isin(keys)]
df['Title'].value_counts()

In [ ]:
def chane_titles(x):
    x = x.strip()
    if x == 'Senior Java Developer':
        return 'Java Developer'
    elif x == 'Senior Software Engineer':
        return 'Software Engineer'
    elif x == 'Senior QA Engineer':
        return 'Software QA Engineer'
    elif x == 'Senior Software Developer':
        return 'Senior Web Developer'
    elif x =='Senior PHP Developer':
        return 'PHP Developer'
    elif x == 'Senior .NET Developer':
        return '.NET Developer'
    elif x == 'Senior Web Developer':
        return 'Web Developer'
    elif x == 'Database Administrator':
        return 'Database Admin/Dev'
    elif x == 'Database Developer':
        return 'Database Admin/Dev'

    else:
        return x
        
    
df['Title'] = df['Title'].apply(chane_titles)
df['Title'].value_counts()

In [ ]:
df["Combined"] = df.RequiredQual + df.JobDescription + df.JobRequirment
df.Combined = df.Combined.apply(lambda x: x.replace("\r\n"," "))
df

In [ ]:
df.iloc[0,4]

In [ ]:
df.to_csv("Modified.csv",index=False)

In [ ]:
class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

In [ ]:
def build_vocab(df, threshold=3):
    """Build a simple vocabulary wrapper."""
    counter = Counter()
    
    for i in range(len(df)):
        caption = df.iloc[i,4]
        tokens = nltk.tokenize.word_tokenize(str(caption))
        counter.update(tokens)

        if (i+1) % 1000 == 0:
                print("[{}/{}] Tokenized the sentences.".format(i+1, len(df)))

    # If the word frequency is less than 'threshold', then the word is discarded.
    words = [word for word, cnt in counter.items() if cnt >= threshold]

    # Create a vocab wrapper and add some special tokens.
    vocab = Vocabulary()
    vocab.add_word('<pad>')
    vocab.add_word('<start>')
    vocab.add_word('<end>')
    vocab.add_word('<unk>')

    # Add the words to the vocabulary.
    for i, word in enumerate(words):
        vocab.add_word(word)
    return vocab

In [ ]:
v = build_vocab(df)
dump(v, open('vocab.pkl', 'wb'))
len(v)

In [ ]:
le = LabelEncoder()
df["TitleUse"] = le.fit_transform(df.Title)
df

In [ ]:
df.iloc[:,5].nunique()

In [ ]:
x = torch.Tensor(np.array(df.iloc[:,5]))
x

In [ ]:
class Data(Dataset):
    def __init__(self,df,vocab):
        self.df = df
        self.vocab = vocab

    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        tokens = nltk.tokenize.word_tokenize(str(df.iloc[idx,4]))
        caption = []
        caption.append(self.vocab('<start>'))
        caption.extend([self.vocab(token) for token in tokens])
        caption.append(self.vocab('<end>'))
        return caption,x[idx]

def collate_fn(data):
    data.sort(key=lambda x: len(x[0]), reverse=True)
    captions,labels = zip(*data)
    lengths = [len(cap) for cap in captions]
    targets = torch.zeros(len(captions), max(lengths)).long()
    for i, cap in enumerate(captions):
        end = lengths[i]
        targets[i, :end] = torch.Tensor(cap[:end])        
    return targets.to(device), labels

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df.Combined,df.Title,test_size = 0.15,random_state = 0)
train = Data(pd.DataFrame(X_train),v)
test = Data(pd.DataFrame(X_test),v)
dataloaderTrain = DataLoader(train,4,num_workers=0,collate_fn=collate_fn)
dataloaderTest = DataLoader(test,4,num_workers=0,collate_fn=collate_fn)

In [ ]:
for i,j in dataloaderTrain:
    print(i)
    print(j)
    print(i.shape,torch.stack(j).shape)
    break

In [ ]:
class Model(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(Model, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True,bidirectional = True, dropout= 0.3)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(hidden_size*2,256)
        self.linear3 = nn.Linear(256,19)
        
    def forward(self,captions):
        embeddings = self.embed(captions)
        hiddens, _ = self.lstm(embeddings)
        x = self.relu(self.linear(hiddens[:,-1,:]))
        outputs = self.linear3(x)
        return outputs

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence
def train(model,data_loader,data_loaderTest,learning_rate,num_epochs):  

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    total_step = len(data_loader)
    print("Total steps are: ", total_step)
    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        for i,(captions, lengths) in enumerate(data_loader):
            # Set mini-batch dataset
            captions = captions.to(device)
            # Forward, backward and optimize
            outputs = model(captions)
            loss = criterion(outputs.to(device),torch.stack(lengths).type(torch.LongTensor).to(device))
            model.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if i % 50 == 0:
                print('Epoch [{}/{}], Step [{}/{}],Training Loss: {:.4f}'
                    .format(epoch, num_epochs, i, total_step, loss.item()))
        correct = 0
        total = 0
        model.eval()
        with torch.no_grad():
            for data in data_loader:
                captions, labels = data
                outputs = model(captions)
                _, predicted = torch.max(outputs.data, 1)
                total += torch.stack(labels).type(torch.LongTensor).size(0)
                correct += (predicted == torch.stack(labels).type(torch.LongTensor).to(device)).sum().item()
        print("Loss after epoch {} is {} and Accuracy is {}".format(epoch,running_loss/total_step,100 * correct / total))
        torch.save(model.state_dict(),"epoch"+str(epoch)+".pb")
        
        correct = 0
        total = 0
        with torch.no_grad():
            for data in data_loaderTest:
                captions, labels = data
                outputs = model(captions)
                _, predicted = torch.max(outputs.data, 1)
                total += torch.stack(labels).type(torch.LongTensor).size(0)
                correct += (predicted == torch.stack(labels).type(torch.LongTensor).to(device)).sum().item()
        print("Loss after epoch {} is {} and Accuracy is {}".format(epoch,running_loss,100 * correct / total))

In [ ]:
model = Model(1024,512,len(v),3).to(device)

# Uncomment below code for training

In [ ]:
# train(model,dataloaderTrain,dataloaderTest,0.001,45)

<h1>Best overall accuracy at epoch 43 </h1>
<h1>training = 99.73457199734572% , testing = 99.62406015037594% </h1>


<h1>Best testing accuracy at epoch 29 </h1>
<h1>training = 98.4737889847379% , testing = 100.0% </h1>

In [ ]:
# torch.save(model.state_dict(), "epoch8.pb")
#torch.save(model.state_dict(), "epoch8.pb")